This code is meant to run mianalyzer headless. Mianalyzer is a code that can run a variety of neural network architrecture and backbones, without the user having to define them himeself. It normally runs as a GUI application. However, when run on a remote server, the GUI version is much too slow,  The mianalyzer author (koerber) kindly provided a python script to run the training sequence  headless, after loading a "pickled"dl object from a pkl file. 

Inputs:
    1. Directory that holds the pkl file
    2. Directory where the model will be saved
    3. The name of the saved pkl file (the pkl extensionis assumed)
    4. The directory containing the training data. The Segmentation_labels subdiretory must also be present.
    5. The prediction folder containing images on which the trained model will be tested. 
    6. model backbone (default is densenet201)
    7. number of epochs (default defined in the pkl file)

 Outputs:
    1. The dl object that will be used to train the model
    2. The trained model (a .h5 file, and a .csv file with the loss history)
    3. A subdirectory that will hold the segmented images.

This Jupyter script is based on code provided by mianalzer author Nils Koerber (nils.koerber@bfr.bund.de)

Modifed by Aryeh Weiss
Last modified: 19 June 2023

This version of the script usees tkinter to prompt for teh various directories and files that are needed.
The defaults are the same, and cliking ok will select default values.
When run on a remote GPU server, this version will be slower because it opens X windows on the local machine,
over the network.


In [ ]:
import pickle
import os 
import cv2
import glob
import matplotlib.pyplot as plt

from pprint import pprint

import tkinter as tk
from tkinter import *
from tkinter import filedialog as fd

from tensorflow.config import list_physical_devices # To see if we're using GPU
# Making sure we're using GPU
print(list_physical_devices())
print("Num GPUs Available: ", len(list_physical_devices('GPU')))

# select one of the multiple GPU cards on the server.
gpuNumber = input("Select GPU or enter for 0\n") or '0'

os.environ["CUDA_VISIBLE_DEVICES"]=gpuNumber  # set GPU if multiple present




In [ ]:
'''
This code is developed and run on at least two machines
Here we setup the default paths for each machine.
Eventually we must find a more general way to do this.
'''

host = os.getenv('HOSTNAME')
if host==None:      # pop os dees not have HOSTNAME defined by default.
    host = "pop"
print(host)

if "dsigpu" in host:
    pklDir = '/home/dsi/aryeh/data/mia/pkl/'
    modelDir = '/home/dsi/aryeh/data/mia/trained_models/'
    modelname = "unetDensnet201_2023-01-24_08-17-37"
    trainingdata = '/home/dsi/aryeh/data/plants/unCropped/resized/'
    predictionFolder = '/home/dsi/aryeh/data/plants/Harvest8Orange5_7Oct17/'
    os.environ['DISPLAY']='localhost:10.0'

elif "pop" in host:
    pklDir = '/media/amw/TOSHIBA EXT/alerding/models/pkl/'
    modelDir = '/media/amw/TOSHIBA EXT/alerding/models/gpuModels/'
    modelname = "unetDensnet201_2023-01-24_08-17-37"
    trainingdata =  '/media/amw/TOSHIBA EXT/alerding/annotated/trainingSet/'
    predictionFolder = '/media/amw/TOSHIBA EXT/alerding/Harvest 8 Orange 5,7 Oct 17.r/vertical/'
else:
    pklDir = None
    modelDir = None 
    modelname = "unetDensnet201_2023-01-24_08-17-37"
    trainingdata = None
    predictionFolder = None

In [ ]:
'''
Put this in a separate cell so that multiple instances of Tk are not invoked
'''
root = Tk()
root.withdraw()

In [ ]:

pklDir = fd.askdirectory(title = "select pkl directory",initialdir=pklDir) + os.sep
modelDir = fd.askdirectory(title = "select model directory", initialdir=modelDir) + os.sep
trainingdata = fd.askdirectory(title = "select training directory",initialdir=trainingdata) + os.sep
predictionFolder = fd.askdirectory(title = "select prediction directory", initialdir=predictionFolder) + os.sep



modelFilePath = fd.askopenfilename(
        title='Open a pkl file',
        initialdir=pklDir)

filename = os.path.basename(modelFilePath)
modelname = os.path.splitext(filename)[0]

print("model: ", modelname)

validationdata = None

loadweights = None



In [ ]:

#Set the window geometry
root.geometry("200x200")
root.deiconify()

# Create object
# root = Tk()

# Adjust size
# root.geometry( "200x200" )

# Change the label text
def show():
	label.config( text = clicked.get() )

# list of backbones
backbones = [None, 'resnet18', 'resnet34','resnet50', 'resnet101', 'resnet152',
             'seresnet18', 'seresnet34', 'seresnet50', 'seresnet101', 'seresnet152', 'seresnext101',
             'senet154','resnext50', 'resnext101', 'vgg16', 'vgg19',
             'densenet121', 'densenet169', 'densenet201', 'inceptionresnetv2', 'inceptionv3',
             'mobilenet', 'mobilenetv2',
             'efficientnetb0','efficientnetb1','efficientnetb02','efficientnetb3','efficientnetb4','efficientnetb5','efficientnetb6','efficientnetb7']

# datatype of menu text
clicked = StringVar()

# Create Dropdown menu
drop = OptionMenu( root , clicked , *backbones )
drop.pack()

# Create Label
label = Label( root , text = "Choose backbone" )
label.pack()

Button(root, text="Quit", command=root.destroy).pack() 
  
# Execute tkinter
root.mainloop()

BackBone = clicked.get()
print(BackBone)

strEpochs = input("input number of epochs or enter to use the predefined value in a pkl file\n") or '0'
epochs = int(strEpochs)

In [ ]:
'''
This is from the version that reads the dl object from a pkl file.
'''

print('load settings')
#filehandler = open(modelname + '.pkl', 'rb')
filehandler = open(pklDir + modelname + '.pkl', 'rb')

dl = pickle.load(filehandler)

if epochs != 0:
    dl.epochs = epochs
if BackBone != None:
    dl.Mode.backbone = BackBone
pprint(vars(dl.Mode))
print(dl.epochs)


In [ ]:
'''
Run this cell if you want to save the dl object as a pkl file
'''

pklFileHandler = open(pklDir+dl.Mode.architecture+'_'+dl.Mode.backbone+'_ep'+str(dl.epochs) + '.pkl','wb')
#import sys
#print(sys.getrecursionlimit())
#sys.setrecursionlimit(4*sys.getrecursionlimit())
#print(sys.getrecursionlimit())

hed = dl.hed
dl.hed = None
model = dl.Model
dl.Model = None
pickle.dump(dl, pklFileHandler)
dl.hed = hed
dl.Model = model



In [ ]:
# pod, stem, background, and unlabeled (for deleaved soybean plants)
numClasses =4

print('init model')
dl.initModel(numClasses)

dl.Model = dl.Mode.getModel(numClasses, 3)

print('load model')
if loadweights is not None:
    dl.Model.load_weights(loadweights)

print(dl.initialized)

In [ ]:
'''
Here we  do the training
'''
        
if dl.initialized:
    print('start training')
    dl.initData_StartTraining(trainingdata, validationdata)
    print('training finished')
else:
    print('could not initialize model')
            
print('saving weights')
modelname=dl.Mode.architecture+'_'+dl.Mode.backbone
modelPath = modelDir+modelname+'_ep'+str(dl.epochs) + '.h5'
print('model path: ', modelPath)
dl.Model.save_weights(modelPath)
print('saving training data')
dl.saveTrainingRecord(modelDir+modelname+'_ep'+str(dl.epochs) +'.csv')
#   print('saving model')
#   dl.Model.save(modelDir+'full')
    



In [ ]:
'''
This cell does a prediction on a directory of jpg images (could be any type, but that is what I have).
predictionFolder contains the images which ill be predicted, and a subdirectory with the modelname will be created
to hold the predicted segmentation.

A h5 file with pretrained weights can be loaded, in which case the traiing step above can be skipped.
However, the previous cells tha set up the dl object must be run. I have not yet succeeded in defining 
a properly working dl object wihtout first usin a saved pkl file.

To use the model that was just trained, press "cancel" when prompted for a weights file.

'''


modelWeightsPath = fd.askopenfilename(
        title='Open a weights file (h5)',
        initialdir=modelDir)

print(modelWeightsPath)

if modelWeightsPath != "":
    dl.Model.load_weights(modelWeightsPath)

predictionFolder = fd.askdirectory(initialdir=predictionFolder) + os.sep
outputPath = predictionFolder + modelname+'_ep'+str(dl.epochs)+'/'
try:
    os.mkdir(outputPath)
except FileExistsError:
    pass
files = glob.glob(predictionFolder + '*.jpg')
print(len(files))



# If we do not want to process the entire folder, we can limit the number of processed image to count. 
# If the entire folder is processed, then it wold be agood idea to comment out the image display 
count = 0
for i in files:
    print(i)
    img = cv2.imread(i)
    prediction = dl.Mode.PredictImage(img)
    plt.figure(count)
    plt.imshow(prediction)
    plt.show()
    cv2.imwrite(outputPath+'segmented_'+os.path.basename(i).replace('jpg', 'png'), prediction)
    count += 1
#    if count > 10:
#       break
    



In [ ]:
print(predictionFolder)